## Preprocessing

In [154]:
#T.Bradford
# October 2021

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [155]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN"],1)

#Name is required Accuracy mid 40% with out
#application_df = application_df.drop(["NAME"],1)
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [156]:
# Determine the number of unique values in each column.

application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [157]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [158]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

#  everything under the 500 classified as other 
application_types_to_replace = list(application_counts[application_counts < 500].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [159]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C4120        1
C2561        1
C1820        1
C2600        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [160]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts >= 10]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
Name: CLASSIFICATION, dtype: int64

In [161]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

#  everything under the 500 classified as other
classifications_replace = list(class_counts[class_counts < 500].index)
# Replace in dataframe
for cls in classifications_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [162]:
# Look at NAME value counts for binning
name_counts = application_df.NAME.value_counts()
#  How many name counts are greater than 5?
name_counts[name_counts>5]


replace_application = list(name_counts[name_counts <= 5].index)

In [163]:
# Determine which values to replace if counts are less than 6.
replace_application = list(name_counts[name_counts < 5].index)

# Replace in dataframe
for app in replace_application:
    application_df.NAME = application_df.NAME.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.NAME.value_counts()

Other                                     19803
PARENT BOOSTER USA INC                     1260
TOPS CLUB INC                               765
UNITED STATES BOWLING CONGRESS INC          700
WASHINGTON STATE UNIVERSITY                 492
                                          ...  
INTERNATIONL TRANSPLANT NURSES SOCIETY        5
ROYAL ORDER OF JESTERS                        5
ARROW CHILD AND FAMILY MINISTRIES             5
SIGMA NU FRATERNITY                           5
RURITAN NATIONAL CORP                         5
Name: NAME, Length: 403, dtype: int64

In [164]:
#  everything under 6 classified as other
classifications_replace = list(name_counts[name_counts < 5].index)
# Replace in dataframe
for cls in classifications_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [165]:
# Convert categorical data to numeric with `pd.get_dummies`

application_df = application_df.drop(["STATUS"],1)
#application_df = application_df.drop(["ASK_AMT"],1)
y = application_df["IS_SUCCESSFUL"]

# One-hot encoding data
X2 = application_df.drop(["IS_SUCCESSFUL"], axis='columns')
X = pd.get_dummies(X2)
X

,ASK_AMT,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,5000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,5000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,5000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [166]:
#merge encoded data with application data
application_df = application_df.merge(X,left_index=True, right_index=True)

#application_df = application_df.drop(application_cat)
print(application_df.info())

application_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Columns: 456 entries, NAME to SPECIAL_CONSIDERATIONS_Y
dtypes: int64(3), object(8), uint8(445)
memory usage: 17.4+ MB
None


,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT_x,IS_SUCCESSFUL,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,N,5000,1,...,0,0,0,0,0,0,0,0,1,0
1,Other,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1,...,1,0,0,0,0,0,0,0,1,0
2,Other,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0,...,0,0,0,0,0,0,0,0,1,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1,...,0,1,0,0,0,0,0,0,1,0
4,Other,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1,...,0,0,1,0,0,0,0,0,1,0


In [167]:
# find obect data types
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
print(application_cat)

['NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


In [168]:
#drop obect data types from dataframe 
application_df = application_df.drop(["NAME"],1)
application_df = application_df.drop(["APPLICATION_TYPE"],1)
application_df = application_df.drop(["AFFILIATION"],1)
application_df = application_df.drop(["CLASSIFICATION"],1)
application_df = application_df.drop(["USE_CASE"],1)
application_df = application_df.drop(["ORGANIZATION"],1)
application_df = application_df.drop(["INCOME_AMT"],1)
application_df = application_df.drop(["SPECIAL_CONSIDERATIONS"],1)
application_df.head()

,ASK_AMT_x,IS_SUCCESSFUL,ASK_AMT_y,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,5000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,108590,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,5000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,6692,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,142590,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [169]:
#drop ASK_AMT_y extra from get_dummies
application_df = application_df.drop(["ASK_AMT_y"],1)

#rename ASK_AMT_x renamed when used get_dummies
application_df.rename(columns={'ASK_AMT_x': 'ASK_AMT'}, inplace=True)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [170]:
#convert string to numberic was getting  ValueError: could not convert string to float:

# X1 = application_df.drop(["STATUS"],1)
#X = application_df.apply(pd.to_numeric, errors='coerce')
#1 = application_df.apply(pd.to_numeric, errors='coerce')


#drop categorical data and separate target
X = application_df.drop(["IS_SUCCESSFUL"], axis='columns').values

y = application_df["IS_SUCCESSFUL"].values


#Dropping 
print(application_df)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

        ASK_AMT  IS_SUCCESSFUL  NAME_AACE INTERNATIONAL  \
0          5000              1                        0   
1        108590              1                        0   
2          5000              0                        0   
3          6692              1                        0   
4        142590              1                        0   
...         ...            ...                      ...   
34294      5000              0                        0   
34295      5000              0                        0   
34296      5000              0                        0   
34297      5000              1                        0   
34298  36500179              0                        0   

       NAME_ACE MENTOR PROGRAM OF AMERICA INC  NAME_ACTS MINISTRY  \
0                                           0                   0   
1                                           0                   0   
2                                           0                   0   
3              

In [171]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [172]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow.keras.layers import LeakyReLU, Dense

number_input_features = len(X_train[0])
print(number_input_features)
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()
leaky_relu = LeakyReLU(alpha=0.001)

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=leaky_relu))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# ADD third Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

446
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 100)               44700     
_________________________________________________________________
dense_25 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_26 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 50,271
Trainable params: 50,271
Non-trainable params: 0
_________________________________________________________________


In [173]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [174]:
 #Create a callback that saves the model's weights every epoch

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint
# Define the checkpoint path and filenames
os.makedirs("weights/",exist_ok=True)
weights_path = "weights/weights.{epoch:08d}.h5"

cp_callback = ModelCheckpoint(
    filepath=weights_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

In [175]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 869us/step - loss: 0.4937 - accuracy: 0.7717

Epoch 00001: saving model to weights\weights.00000001.h5, period=5
Epoch 2/100
804/804 [==============================] - 1s 857us/step - loss: 0.4410 - accuracy: 0.7952

Epoch 00002: saving model to weights\weights.00000002.h5, period=5
Epoch 3/100
804/804 [==============================] - 1s 832us/step - loss: 0.4314 - accuracy: 0.7967

Epoch 00003: saving model to weights\weights.00000003.h5, period=5
Epoch 4/100
804/804 [==============================] - 1s 832us/step - loss: 0.4264 - accuracy: 0.7985

Epoch 00004: saving model to weights\weights.00000004.h5, period=5
Epoch 5/100
804/804 [==============================] - 1s 824us/step - loss: 0.4236 - accuracy: 0.7984

Epoch 00005: saving model to weights\weights.00000005.h5, period=5
Epoch 6/100
804/804 [==============================] - 1s 827us/step - loss: 0.4216 - accuracy: 0.8007

Epoch 00006: saving model to weights\weig

In [176]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4477 - accuracy: 0.7915
Loss: 0.44773539900779724, Accuracy: 0.7914868593215942


In [177]:
# Export our model to HDF5 file
nn.save("HDf5_file/AlphabetSoupCharity.h5")